In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA



In [2]:
data= pd.read_excel('C:\\Users\\arozhkov\\Desktop\\Python\\280425\\Список точек 12.2024.xlsx', sheet_name='Лист1')

In [3]:
data.rename(columns={'Аптечная сеть': 'Название сети', 'Товарооборот Alpharm': 'Товарооборот'}, inplace=True)

In [4]:
data= data.dropna()

In [5]:
data = data.sample(n=5000, random_state=1)  # random_state для воспроизводимости

In [6]:

df = pd.DataFrame(data)


# Кодируем категориальные признаки
label_encoder = LabelEncoder()
df['Название сети_код'] = label_encoder.fit_transform(df['Название сети'])
df['Населенный пункт_код'] = label_encoder.fit_transform(df['Населенный пункт'])

# Выбираем признаки для кластеризации
features = df[['Название сети_код', 'Товарооборот', 'Населенный пункт_код']]

# Масштабируем данные
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

In [ ]:
%%time
wcss = []
n_samples = len(scaled_features)  # Получаем количество образцов
for i in range(1, n_samples + 1):  # Изменяем диапазон до n_samples + 1
    kmeans = KMeans(n_clusters=i, init='k-means++', random_state=42)
    kmeans.fit(scaled_features)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, n_samples + 1), wcss)
plt.title('Метод локтя')
plt.xlabel('Количество кластеров')
plt.ylabel('WCSS')
plt.show()

In [ ]:
# Выбираем количество кластеров (например, 3)
n_clusters = 3

# Обучаем модель K-Means
kmeans = KMeans(n_clusters=n_clusters, init='k-means++', random_state=42)
clusters = kmeans.fit_predict(scaled_features)

# Добавляем кластеры в исходный DataFrame
df['Кластер'] = clusters

# Визуализируем результаты с помощью PCA
pca = PCA(n_components=2)
principal_components = pca.fit_transform(scaled_features)
principal_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'])
principal_df['Кластер'] = clusters

plt.figure(figsize=(8, 6))
for i in range(n_clusters):
    plt.scatter(principal_df[principal_df['Кластер'] == i]['PC1'],
                principal_df[principal_df['Кластер'] == i]['PC2'],
                label=f'Кластер {i}')

plt.title('Кластеризация аптечных точек')
plt.xlabel('Главная компонента 1')
plt.ylabel('Главная компонента 2')
plt.legend()
plt.show()

# Выводим результаты
print(df[['Название сети', 'Товарооборот', 'Населенный пункт', 'Кластер']])
